In [1]:
import os
import wandb
from dotenv import load_dotenv

# load variables from env
load_dotenv()

# Start weights and biases
wandb.login()


wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from WANDB_API_KEY.
wandb: Currently logged in as: coffeedrunk to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
import tensorflow as tf

# Enable memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ Enabled memory growth for {len(gpus)} GPU(s)")
    except RuntimeError as e:
        print(e)

# Prevent Out Of Memory
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

2026-02-20 09:15:05.677576: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-20 09:15:05.715187: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9360] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-02-20 09:15:05.715218: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-02-20 09:15:05.715243: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1537] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-02-20 09:15:05.729506: I tensorflow/core/platform/cpu_feature_g

✅ Enabled memory growth for 1 GPU(s)


2026-02-20 09:15:07.234079: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2026-02-20 09:15:07.257023: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2026-02-20 09:15:07.258891: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
# Read the dataset
from utils.data import load_data

X_train, y_train, X_test, y_test, labels = load_data()
n_labels = len(labels)

2026-02-20 09:15:07.309965: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2026-02-20 09:15:07.311936: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2026-02-20 09:15:07.313632: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

## Guaxinim Cansado CNN (RaccoonCNN)

In [ ]:
from utils.config import WandbConfig, ExperimentConfig

config = ExperimentConfig(input_shape=(32,32,3),
                          epochs=20,
                          batch_size=32,
                          model_name="GuaxinimCNN",
                          normalize=False)

experiment_name = f"{config.model_name}_{config.input_shape[0]}_{config.batch_size}"

wandb_config = WandbConfig(project_name="deep-learning-cifar10-classification_custom",
                           experiment_name=experiment_name)


In [ ]:
from utils.train import run_experiment

(train_ds,
 val_ds,
 test_ds,
 model,
 history,
 run) = run_experiment(X_train, y_train,
               X_test, y_test,
               wandb_config=wandb_config,
               config=config,
               class_names = labels)


## Testing parameters and models: Custom CNNs

In [ ]:
from utils.config import WandbConfig, ExperimentConfig

input_arr = [32, 64, 224]
batch_arr = [32,64,128]
model_arr = ["GuaxinimCNN", "baseline_cnn", "deeper_cnn"]

for size in input_arr:
    for batch in batch_arr:
        for model_name in model_arr:

            try: 
                config = ExperimentConfig(input_shape=(size,size,3),
                                        epochs=30,
                                        batch_size=batch,
                                        model_name=model_name,
                                        normalize=True)

                experiment_name = f"{config.model_name}_{config.input_shape[0]}_{config.batch_size}_norm"

                wandb_config = WandbConfig(project_name="deep-learning-cifar10-classification_custom",
                                        experiment_name=experiment_name)


                from utils.train import run_experiment

                (train_ds,
                val_ds,
                test_ds,
                model,
                history,
                run) = run_experiment(X_train, y_train,
                            X_test, y_test,
                            wandb_config=wandb_config,
                            config=config,
                            class_names = labels)

            except:
                print(f"Could not run with the parameters: {size}, {batch}, {model_name}")

## Testing parameters and models: Transfer Learning

In [4]:
from utils.config import WandbConfig, ExperimentConfig
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# input_arr = [64, 96, 224]
input_arr = [224]
# batch_arr = [64,128]
batch_arr = [128]
lr_arr = [1e-3]
model_arr = ["MobileNetV2", "ConvNeXtTiny"]

for size in input_arr:
    for batch in batch_arr:
        for model_name in model_arr:
            for lr in lr_arr:

                try: 
                    config = ExperimentConfig(input_shape=(size,size,3),
                                            epochs=20,
                                            batch_size=batch,
                                            model_name=model_name,
                                            normalize=False,
                                            learning_rate=lr,
                                            augment=False)

                    experiment_name = f"{config.model_name}_{config.input_shape[0]}_{config.batch_size}"

                    wandb_config = WandbConfig(project_name="deep-learning-cifar10-classification_final",
                                            experiment_name=experiment_name)


                    from utils.train import run_experiment

                    if model_name == "MobileNetV2":
                        preprocess_fn = preprocess_input

                    else:
                        preprocess_fn = None

                    (train_ds,
                    val_ds,
                    test_ds,
                    model,
                    history,
                    run) = run_experiment(X_train, y_train,
                                X_test, y_test,
                                wandb_config=wandb_config,
                                config=config,
                                class_names = labels,
                                preprocess_fn=preprocess_fn)

                except:
                    print(f"Could not run with the parameters: {size}, {batch}, {model_name}, {lr}")

/usr/local/lib/python3.10/dist-packages/pydantic/main.py:308: UserWarning: Pydantic serializer warnings:
  Expected `list[str]` but got `tuple` - serialized value may not be as expected
  Expected `list[str]` but got `tuple` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


2026-02-20 09:15:14.455475: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:231] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 12.0
2026-02-20 09:15:14.455489: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:234] Used ptxas at ptxas
2026-02-20 09:15:14.455543: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2026-02-20 09:15:14.456949: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2026-02-20 09:15:14.456970: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2026-02-20 09:15:14.456987: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:19

Epoch 1/20


2026-02-20 09:15:20.249383: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8907
2026-02-20 09:16:00.357359: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.


  1/313 [..............................] - ETA: 4:44:18 - loss: 2.5010 - accuracy: 0.1094

...

/usr/local/lib/python3.10/dist-packages/pydantic/main.py:308: UserWarning: Pydantic serializer warnings:
  Expected `list[str]` but got `tuple` - serialized value may not be as expected
  Expected `list[str]` but got `tuple` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


Epoch 1/20


2026-02-20 09:20:08.002059: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2026-02-20 09:20:08.076024: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2026-02-20 09:20:08.076049: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2026-02-20 09:20:08.076063: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2026-02-20 09:20:08.076424: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2026-02-20

313/313 [==============================] - 140s 374ms/step - loss: 0.3924 - accuracy: 0.8776 - val_loss: 0.2547 - val_accuracy: 0.9155
Epoch 2/20
313/313 [==============================] - 110s 353ms/step - loss: 0.2267 - accuracy: 0.9243 - val_loss: 0.2251 - val_accuracy: 0.9261
Epoch 3/20
313/313 [==============================] - 110s 352ms/step - loss: 0.1963 - accuracy: 0.9336 - val_loss: 0.2156 - val_accuracy: 0.9286
Epoch 4/20
313/313 [==============================] - 110s 352ms/step - loss: 0.1750 - accuracy: 0.9414 - val_loss: 0.2159 - val_accuracy: 0.9266
Epoch 5/20
313/313 [==============================] - 110s 352ms/step - loss: 0.1580 - accuracy: 0.9467 - val_loss: 0.2090 - val_accuracy: 0.9304
Epoch 6/20
313/313 [==============================] - 110s 352ms/step - loss: 0.1416 - accuracy: 0.9530 - val_loss: 0.2132 - val_accuracy: 0.9305
Epoch 7/20
313/313 [==============================] - 110s 352ms/step - loss: 0.1277 - accuracy: 0.9576 - val_loss: 0.2062 - val_accura

epoch/accuracy,▁▄▅▅▆▆▆▇▇▇██
epoch/epoch,▁▂▂▃▄▄▅▅▆▇▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▄▄▃▃▃▂▂▂▁▁▁
epoch/val_accuracy,▁▅▆▆▇▇█▇▇▇█▇
epoch/val_loss,█▄▂▂▁▂▁▃▂▃▃▅
final_val_accuracy,▁
final_val_loss,▁
epoch/accuracy,0.9811
epoch/epoch,11
epoch/learning_rate,0.001
